In [1]:

import pandas as pd
import re #regex library to format fast to read into pd
import os

import matplotlib.pyplot as plt
import feather

import numpy as np
import platform
import mne


from mne.channels import find_ch_adjacency, make_1020_channel_selections
from mne.stats import spatio_temporal_cluster_test
from mne.stats import (ttest_1samp_no_p, bonferroni_correction, fdr_correction,


                       permutation_t_test, permutation_cluster_1samp_test,spatio_temporal_cluster_1samp_test)
import sys
if platform.system()=='Darwin':
    os.chdir('/Volumes/BBC/BBC/WP1/data/EEG/tsk/')
    #sys.path.append('/Users/leupinv/BBC/WP1/data/Code/python/BBC')
    #if this doesn't work pound line right above this, restart the kernel, rerun the cell.
    #Then uncomment the same line and rerun

else:
    #os.chdir('Z:/BBC/WP1/data/EEG/tsk')
    #sys.path.append('C:/Users/Vivi/switchdrive/BBC/WP1/data/Code/python/BBC')
    os.chdir('Z:/BBC/WP1/data/EEG/tsk')

from base.files_in_out import getListOfFiles,GetFiles
from stats.stats_helper import filter_list, tTest_ana
import stats.t_tests_cluster as clus
import stats.stats_helper as s_hp
import base.base_constants as cs

C:\Users\Vivi\anaconda3\envs\mne\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
C:\Users\Vivi\anaconda3\envs\mne\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.5.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
%load_ext autoreload
%autoreload 2
import matplotlib as mpl
mpl.rcParams['svg.fonttype']='none'
mpl.rcParams['text.usetex']=False
plt.style.use('default')

mpl.rcParams['svg.fonttype']='none'
mpl.rcParams['text.usetex']=False

In [40]:
def data_cutter(data, time_limits=[-0.3, 0.8], new_time_limits=[-0.2, 0.6], sfreq=256):
    """
    Cuts a 3D EEG data array ( time x electrodes) based on specified time limits.
    Parameters:
        data (np.ndarray): EEG data of shape (electrodes, time)
        time_limits (list of float): Original epoch time range in seconds (e.g., [-0.3, 0.8])
        new_time_limits (list of float): Desired time range to extract (e.g., [0.2, 0.6])
        sfreq (int): Sampling frequency in Hz (default is 256)
    Returns:
        np.ndarray: The cut data with shape (new_time_points,electrodes)
    """
    # Calculate the sample indices corresponding to the new time limits
    start_idx = int(np.round((new_time_limits[0] - time_limits[0]) * sfreq))
    end_idx = int(np.round((new_time_limits[1] - time_limits[0]) * sfreq))
    if np.size(data.shape) > 1:
        # Cut the data along the time dimension (axis=1)
        return data[start_idx:end_idx,:]
    else:
        return data[start_idx:end_idx]



In [30]:
np.size(data.shape)

2

# Compute gavgs

In [4]:
g_num='g01'
file_dir=f'ana/deconvolution/ep_betas/modfit/dc/{g_num}_modfit_hep_awareness_unaware_dc.ep'
f = open(file_dir)


In [5]:
data=np.loadtxt(file_dir)
data.shape

(256, 128)

In [18]:
subject_list=cs.G_N
X=np.empty((len(subject_list),data.shape[1],data.shape[0]))
cond_list=["awareness_unaware","awareness","awareness","card_phase_sys","_Intercept_","awareness_unaware_card_phase_sys"]
fit='base'
dc='dc'
beta_type=cond_list[4]

for idx,g_num in enumerate(subject_list):
    file_dir=f'ana/deconvolution/ep_betas/{fit}/{dc}/{g_num}_{fit}_hep_{beta_type}_{dc}.ep'
    data=np.loadtxt(file_dir)

    X[idx,:,:]=data.T




In [19]:
gavg=np.mean(X,0).T
filename=f'ana/deconvolution/ep_betas/{fit}/{dc}/gavg_{fit}_hep_{beta_type}_{dc}.ep'

np.savetxt(filename,gavg)

# For cardfit/awafit

In [6]:
g_num='g01'
file_dir=f'ana/deconvolution/ep_betas/cardfit/dc/{g_num}_sys_hep__Intercept__dc.ep'
f = open(file_dir)


In [7]:
data=np.loadtxt(file_dir)

data.shape

(256, 128)

In [8]:
subject_list=cs.G_N
X=np.empty((len(subject_list),data.shape[1],data.shape[0]))
sys_cond='unaware'
fit='awafit'
dc='dc'
beta_type="_Intercept_"

for idx,g_num in enumerate(subject_list):
    file_dir=f'ana/deconvolution/ep_betas/{fit}/{dc}/{g_num}_{sys_cond}_hep_{beta_type}_{dc}.ep'
    data=np.loadtxt(file_dir)

    X[idx,:,:]=data.T




In [24]:
gavg=np.mean(X,0).T
filename=f'ana/deconvolution/ep_betas/{fit}/{dc}/gavg_{fit}_hep_{sys_cond}_{dc}.ep'

np.savetxt(filename,gavg)

# Create df to get BF in R


# Plot waveforms gavgs
## plot erps

In [4]:

#define variables
modtype='rawerp'
erp_type="hep"
cond='unaware'
dc_type="nodc"


filepath='ana/MNE/evo_list/maskOFF/correct/cfa/hep/norm/'
filename=f"maskOFF_correct_cfa_hep_norm_RRCA_list-ave.fif"
fp=filepath+filename
evo_awa=mne.read_evokeds(fp)

filepath='ana/MNE/evo_list/maskOFF/correct/cfa/hep/norm/'
filename=f"maskOFF_correct_cfa_hep_norm_RRCU_list-ave.fif"
fp=filepath+filename
evo_unawa=mne.read_evokeds(fp)


Reading ana/MNE/evo_list/maskOFF/correct/cfa/hep/norm/maskOFF_correct_cfa_hep_norm_RRCA_list-ave.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 128) active
    Found the data of interest:
        t =    -199.22 ...    1000.00 ms (0.13 × hep/normal/correct/aware/R/exh/RRCA + 0.08 × hep/normal/correct/aware/R/inh/RRCA + 0.47 × hep/normal/correct/aware/R2/exh/RRCA + 0.32 × hep/normal/correct/aware/R2/inh/RRCA
 G_n= g01)
        0 CTF compensation matrices available
        nave = 345 - aspect type = 100
Projections have already been applied. Setting proj attribute to True.
No baseline correction applied
    Read a total of 1 projection items:
        Average EEG reference (1 x 128) active
    Found the data of interest:
        t =    -199.22 ...    1000.00 ms (0.18 × hep/normal/correct/aware/R/exh/RRCA + 0.15 × hep/normal/correct/aware/R/inh/RRCA + 0.42 × hep/normal/correct/aware/R2/exh/RRCA + 0.25 × hep/normal/correct/aware/R2/inh/RRCA
 G_n= g03)
    

In [7]:
evo_awa=[ev.crop(-0.1,0.7) for ev in evo_awa]
evo_unawa=[ev.crop(-0.1,0.7) for ev in evo_unawa]

C:\Users\Vivi\AppData\Local\Temp\ipykernel_12148\1120704418.py:1: RuntimeWarning: tmax is not in Evoked time interval. tmax is set to evoked.tmax (0.699219 sec)
  evo_awa=[ev.crop(-0.1,0.7) for ev in evo_awa]
C:\Users\Vivi\AppData\Local\Temp\ipykernel_12148\1120704418.py:1: RuntimeWarning: tmax is not in Evoked time interval. tmax is set to evoked.tmax (0.699219 sec)
  evo_awa=[ev.crop(-0.1,0.7) for ev in evo_awa]
C:\Users\Vivi\AppData\Local\Temp\ipykernel_12148\1120704418.py:1: RuntimeWarning: tmax is not in Evoked time interval. tmax is set to evoked.tmax (0.699219 sec)
  evo_awa=[ev.crop(-0.1,0.7) for ev in evo_awa]
C:\Users\Vivi\AppData\Local\Temp\ipykernel_12148\1120704418.py:1: RuntimeWarning: tmax is not in Evoked time interval. tmax is set to evoked.tmax (0.699219 sec)
  evo_awa=[ev.crop(-0.1,0.7) for ev in evo_awa]
C:\Users\Vivi\AppData\Local\Temp\ipykernel_12148\1120704418.py:1: RuntimeWarning: tmax is not in Evoked time interval. tmax is set to evoked.tmax (0.699219 sec)
  e

In [8]:
regions=dict(dc_post=['A5','A14','A16','A18','A23'],dc_front=['C32','D4'],nodc=['B12','B14','B23','B24'] )

In [9]:
from mne import pick_channels

region_indices={}

for region_name, region_channels in regions.items():
    indices=pick_channels(evo_awa[0].ch_names, include=region_channels)
    region_indices[region_name]=indices
print(region_indices)

{'dc_post': array([ 4, 13, 15, 17, 22]), 'dc_front': array([95, 99]), 'nodc': array([43, 45, 54, 55])}


b cvvccvcif phy_sig=='rsp_phase':
    colors=[['olivedrab','darkgreen'],['tab:pink','purple']]
elif phy_sig=='cardiac_phase':
    colors=[['c','b'],['tab:orange','r']]
else:
    colors=['k','grey']

In [54]:


evokeds=dict(aware=evo_awa,unaware=evo_unawa)

fig,axs=plt.subplots(1,3,figsize=(10,5))
n=0
for regio, picked in regions.items():
    print(picked)

    fig_fin=mne.viz.plot_compare_evokeds(evokeds,colors=['r','g'],ci=False, picks=picked,show_sensors=True,legend='upper right',truncate_yaxis=False,axes=axs[n],combine='mean',vlines=[0,0.49,0.52,0.7],ylim=dict(eeg=[-1.5,1.5]))
    n+=1


['A5', 'A14', 'A16', 'A18', 'A23']
combining channels using "mean"
combining channels using "mean"
['C32', 'D4']
combining channels using "mean"
combining channels using "mean"
['B12', 'B14', 'B23', 'B24']
combining channels using "mean"
combining channels using "mean"


In [55]:
file_dir=f'ana/deconvolution/figures/waveforms_gavg_{modtype}_{erp_type}_{cond}_{dc_type}.svg'
fig.savefig(file_dir,format='svg')

## plot awafit awa vs unawa

In [9]:

#define variables
modtype='awafit'
erp_type="hep"
cond='aware'
dc_type="dc"

#load corresponding file


file_dir=f'ana/deconvolution/ep_betas/{modtype}/{dc_type}/gavg_{modtype}_{erp_type}_{cond}_{dc_type}.ep'
f = open(file_dir)

In [10]:
data_aware=np.loadtxt(file_dir)
data_aware=data_cutter(data_aware,time_limits=[-0.2,0.8],new_time_limits=[-0.1,0.7])
data.shape

(256, 128)

In [17]:

#define variables
modtype='awafit'

cond='unaware'


#load corresponding file


file_dir=f'ana/deconvolution/ep_betas/{modtype}/{dc_type}/gavg_{modtype}_{erp_type}_{cond}_{dc_type}.ep'
f = open(file_dir)

In [18]:
data_unaware=np.loadtxt(file_dir)
data_unaware=data_cutter(data_unaware,time_limits=[-0.2,0.8],new_time_limits=[-0.1,0.7])


In [19]:
def get_evo(data,tmin=-0.1):
    biosemi_montage = mne.channels.make_standard_montage('biosemi128')

    info = mne.create_info(ch_names=biosemi_montage.ch_names, sfreq=256.,
                           ch_types='eeg')
    evok=mne.EvokedArray(data,info,tmin=tmin)
    evok.set_montage(biosemi_montage)
    return evok

In [20]:
evo_awa=get_evo(data_aware.T)
evo_unawa=get_evo(data_unaware.T)

In [44]:
regions=dict(dc_post=['A5','A14','A16','A18','A23'],dc_front=['C32','D4'],nodc=['B12','B14','B23','B24'] )

regions=dict(nodc_post=['A13','A14','A16','A15','A23'],dc_front=['C32','D4'],nodc=['B12','B14','B23','B24'] )

if phy_sig=='rsp_phase':
    colors=[['olivedrab','darkgreen'],['tab:pink','purple']]
elif phy_sig=='cardiac_phase':
    colors=[['c','b'],['tab:orange','r']]
else:
    colors=['k','grey']

In [45]:
%matplotlib qt

evokeds=dict(aware=evo_awa,unaware=evo_unawa)

fig,axs=plt.subplots(1,3,figsize=(10,5))
n=0
BF_lim=3
for regio, picked in regions.items():
    print(picked)
    file_dir_BF=f'ana/deconvolution/stats/BF_vector_{modtype}_{erp_type}_awareness_{dc_type}_{regio}.ep'
    data_BF=np.loadtxt(file_dir_BF)
    times=evo_awa.times
    data_BF=data_cutter(data_BF,time_limits=[-0.2,0.8],new_time_limits=[-0.1,0.7])
    fig_fin=mne.viz.plot_compare_evokeds(evokeds,ci=False, colors=['r','g'],picks=picked,show_sensors=True,legend='upper right',truncate_yaxis=False,axes=axs[n],combine='mean',vlines=[0,0.49,0.52,0.7],ylim=dict(eeg=[-1.5*1e6,1.5*1e6]),truncate_xaxis=False)
    ax2=axs[n].twinx()
    ax2.plot(times,data_BF,color='gray',lw=1.2,alpha=0.6, label='BF10')
    #ax2.set_yscale('log')
    ax2.set_ylim(0.1,10)
    ax2.axhline(BF_lim,linestyle='--',color='pink',alpha=0.5)
    ax2.fill_between(times,0.1,data_BF,where=data_BF>BF_lim,color='pink',alpha=0.3, label=f'BF > {BF_lim}')
    n+=1


['A5', 'A14', 'A16', 'A18', 'A23']
combining channels using "mean"
combining channels using "mean"
['C32', 'D4']
combining channels using "mean"
combining channels using "mean"
['B12', 'B14', 'B23', 'B24']
combining channels using "mean"
combining channels using "mean"


In [46]:
file_dir=f'ana/deconvolution/figures/waveforms_gavg_{modtype}_{erp_type}_{cond}_{dc_type}_BF.svg'
fig.savefig(file_dir,format='svg')

In [47]:
file_dir=f'ana/deconvolution/figures/waveforms_gavg_{modtype}_{erp_type}_{cond}_{dc_type}_BF.png'
fig.savefig(file_dir,format='png')